In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {live}")
print(f"Client Connected: {connected}")

In [ ]:
import time
from weaviate.classes.query import HybridFusion, QueryReference, MetadataQuery

chunks = client.collections.get("<COLLECTION_NAME>").with_tenant("<TENANT_ID>")

times = []
for i in range(10):
    start = time.time()
    response = chunks.query.hybrid(
        query="<QUERY>",
        target_vector="<NAMED_VECTOR>",
        limit=10,
        alpha=0.25,
        fusion_type=HybridFusion.RELATIVE_SCORE,
        auto_limit=1,
        return_references=QueryReference(
            link_on="<REFERENCE_PROPERTY>",
            return_properties=["<PROPERTY_1>", "<PROPERTY_2>", "<PROPERTY_3>"]
        ),
        return_metadata=MetadataQuery(certainty=True, score=True, explain_score=True)
    )
    elapsed = (time.time() - start) * 1000
    times.append(elapsed)
    print(f"Run {i+1}: {elapsed:.0f}ms")

print(f"\nAvg: {sum(times)/len(times):.0f}ms | Min: {min(times):.0f}ms | Max: {max(times):.0f}ms")

import openai

times = []
for i in range(10):
    start = time.time()
    response = openai.embeddings.create(
        model="<EMBEDDING_MODEL>",
        input="<QUERY>",
        dimensions=<DIMENSIONS>
    )
    elapsed = (time.time() - start) * 1000
    times.append(elapsed)
    print(f"Run {i+1}: {elapsed:.0f}ms")

print(f"\nOpenAI Embedding - Avg: {sum(times)/len(times):.0f}ms | Min: {min(times):.0f}ms | Max: {max(times):.0f}ms")

In [ ]:
# Hybrid Query with explained score
from weaviate.classes.query import MetadataQuery

coll = client.collections.get("<COLLECTION_NAME>")
response = coll.query.hybrid(
    query="<QUERY>",
    max_vector_distance=0.7,
    alpha=0.5,
    limit=3,
    return_metadata=MetadataQuery(score=True, explain_score=True)
    )

for o in response.objects:
    print(o.properties)
    print(o.metadata.score, o.metadata.explain_score)

In [ ]:
# Hybrid Query with distance
coll = client.collections.get("<COLLECTION_NAME>")
response = coll.query.hybrid(query="<QUERY>",max_vector_distance=0.23,limit=3)

for o in response.objects:
    print(o.properties)

In [ ]:
# Hybrid Query with Tenant
coll = client.collections.get("<COLLECTION_NAME>")
tenant_collection = coll.with_tenant("<TENANT_ID>")
response = tenant_collection.query.hybrid(
    query="What is AI",
    limit=3
)

for o in response.objects:
    print(o.properties)

In [ ]:
# Hybrid Query with Tenant in target vector
coll = client.collections.get("<COLLECTION_NAME>")
tenant_collection = coll.with_tenant("<TENANT_NAME>")
response = tenant_collection.query.hybrid(
    query="<QUERY>",
    target_vector="<NAMED_VECTOR>",
    limit=3
)

for o in response.objects:
    print(o.properties)

In [ ]:
# Hybrid Query with explained score
from weaviate.classes.query import MetadataQuery
import re
from pprint import pprint
import json

coll = client.collections.get("<COLLECTION_NAME>")
response = coll.query.hybrid(
    query="<QUERY>",
    query_properties=["<PROPERTY_1>", "<PROPERTY_2>"],
    alpha=0,
    limit=2,
    return_metadata=MetadataQuery(creation_time=True,last_update_time=True, distance=True, certainty=True, score=True, explain_score=True, is_consistent=True)
    )

def print_results(response):
    """
    Weaviate query results
    """
    print("\n" + "="*80)
    print(f"QUERY RESULTS - {len(response.objects)} items found")
    print("="*80)
    
    for i, obj in enumerate(response.objects):
        print(f"\nRESULT #{i+1}")
        print("-"*80)
        
        # Print score information
        print(f"SCORE: {obj.metadata.score:.6f}")
        
        # Extract and print original score
        match = re.search(r'original score ([\d.]+)', obj.metadata.explain_score)
        if match:
            print(f"ORIGINAL SCORE: {float(match.group(1)):.6f}")
        
        # Print explain score on multiple lines for readability
        print(f"EXPLAIN SCORE: {obj.metadata.explain_score}")
        
        # Print other metadata
        print(f"DISTANCE: {obj.metadata.distance if hasattr(obj.metadata, 'distance') else 'N/A'}")
        print(f"CERTAINTY: {obj.metadata.certainty if hasattr(obj.metadata, 'certainty') else 'N/A'}")
        print(f"IS CONSISTENT: {obj.metadata.is_consistent if hasattr(obj.metadata, 'is_consistent') else 'N/A'}")
        
        # Print properties with nice formatting
        print("\nPROPERTIES:")
        for key, value in obj.properties.items():
            # Format the value for better readability
            if isinstance(value, (dict, list)):
                # For complex structures, use pretty printing
                print(f"  {key}:")
                formatted_value = json.dumps(value, indent=4)
                # Indent each line
                for line in formatted_value.split('\n'):
                    print(f"    {line}")
            else:
                # For simple values, print directly
                print(f"  {key}: {value}")
        
        print("-"*80)

# Usage:
print_results(response)

In [ ]:
# Hybrid Query with explained score and performance timing
from weaviate.classes.query import MetadataQuery
import re
from pprint import pprint
import json
import time 

coll = client.collections.get("<COLLECTION_NAME>")

# Measure performance - start timing
start_time = time.time() * 1000  # Convert to milliseconds

response = coll.query.hybrid(
    query="<QUERY>",
    alpha=0.5,
    limit=3,
    return_metadata=MetadataQuery(creation_time=True,last_update_time=True, distance=True, certainty=True, score=True, explain_score=True, is_consistent=True)
    )

# Measure performance - end timing
end_time = time.time() * 1000  # Convert to milliseconds
time_taken = end_time - start_time

def print_results(response, time_taken):
    """
    Weaviate query results with performance timing
    """
    print("\n" + "="*80)
    print(f"QUERY RESULTS - {len(response.objects)} items found")
    print("="*80)
    
    for i, obj in enumerate(response.objects):
        print(f"\nRESULT #{i+1}")
        print("-"*80)
        
        # Print score information
        print(f"SCORE: {obj.metadata.score:.6f}")
        
        # Extract and print original score
        match = re.search(r'original score ([\d.]+)', obj.metadata.explain_score)
        if match:
            print(f"ORIGINAL SCORE: {float(match.group(1)):.6f}")
        
        # Print explain score on multiple lines for readability
        print(f"EXPLAIN SCORE: {obj.metadata.explain_score}")
        
        # Print other metadata
        print(f"DISTANCE: {obj.metadata.distance if hasattr(obj.metadata, 'distance') else 'N/A'}")
        print(f"CERTAINTY: {obj.metadata.certainty if hasattr(obj.metadata, 'certainty') else 'N/A'}")
        print(f"IS CONSISTENT: {obj.metadata.is_consistent if hasattr(obj.metadata, 'is_consistent') else 'N/A'}")
        print(f"CREATION TIME: {obj.metadata.creation_time if hasattr(obj.metadata, 'creation_time') else 'N/A'}")
        print(f"LAST UPDATE TIME: {obj.metadata.last_update_time if hasattr(obj.metadata, 'last_update_time') else 'N/A'}")
        
        # Print performance timing (added after LAST UPDATE TIME)
        print(f"TIME TAKEN: {time_taken:.2f}ms ({time_taken/1000:.3f}s) ({time_taken/60000:.4f}m)")
        
        # Print properties with nice formatting
        print("\nPROPERTIES:")
        for key, value in obj.properties.items():
            # Format the value for better readability
            if isinstance(value, (dict, list)):
                # For complex structures, use pretty printing
                print(f"  {key}:")
                formatted_value = json.dumps(value, indent=4)
                # Indent each line
                for line in formatted_value.split('\n'):
                    print(f"    {line}")
            else:
                # For simple values, print directly
                print(f"  {key}: {value}")
        
        print("-"*80)

# Usage:
print_results(response, time_taken)